https://jira.x5.ru/browse/CVMUPG-1066

In [1]:
name = 'CVMUPG-1066_savushkin_add'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from logic_filters import *
from datetime import timedelta
from spark import *
from utils_osipov import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

In [5]:
experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        77113: "sms_viber_tg",
        77113: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-23',
    'date_communication_to': '2021-12-29',

    'wave_ids': [
        -1 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [6]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [7]:
guests_tg1 = (experiment_guests_communications
          .filter(F.col('group_name') == '0_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [8]:
guests_cg1 = (experiment_guests_communications
          .filter(F.col('group_name') == '1_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [9]:
experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        76934: "sms_viber_tg",
        76934: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2021-12-23',
    'date_communication_to': '2021-12-29',

    'wave_ids': [
        77054 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})
#0_ + camp_wave_id +_sms_viber_ + cg (если номера волн равны) / tg (если номера волн разные)

In [10]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [11]:
guests_tg2 = (experiment_guests_communications
          .filter(F.col('group_name') == '0_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [12]:
guests_cg2 = (experiment_guests_communications
          .filter(F.col('group_name') == '1_' +
                str(list(experiment_params.map_camp_wave_ids_to_suffixes.keys())[0]) + '_' +
                list(experiment_params.map_camp_wave_ids_to_suffixes.values())[0])
          .select('customer_rk').distinct()
         )

In [13]:
dt1 = datetime.datetime.strptime(experiment_params.date_communication_from, '%Y-%m-%d')
dt2 = datetime.datetime.strptime(experiment_params.date_communication_to, '%Y-%m-%d')

In [14]:
start_date = datetime.date(dt1.year, dt1.month, dt1.day)
end_date = datetime.date(dt2.year, dt2.month, dt2.day)

In [15]:
plu_codes = [3367104,3388814,3672506,3919797,3926055,4046891,4046892,4046893,4118966,4137277]

In [16]:
guests_all = (guests_tg1.withColumn('group', F.lit('tg1'))
              .union(guests_tg2.withColumn('group', F.lit('tg2')))
              .union(guests_cg1.withColumn('group', F.lit('cg1')))
              .union(guests_cg2.withColumn('group', F.lit('cg2'))).distinct()
             )

In [5]:
# guests_all.write.parquet('temp1066_guests_all', mode='overwrite')
guests_all = spark.read.parquet('temp1066_guests_all')

### 1. Сегментация на персон откликнувшихся гостей.

In [21]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )

In [22]:
loyalty_cards = (spark
                .table(LOYALTY_CARDS)
                .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                .withColumnRenamed("loyalty_account_id", "account_no")
                .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                .select('account_no', 'loyalty_card_no')
            )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(guests_all, on='customer_rk', how='inner')

In [23]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только доступных госте

In [24]:
checks_items = (spark.table(CHECKS_ITEMS) 
                .withColumnRenamed('plu_id', 'plu_code')
                .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                .withColumnRenamed('discount_amt', 'zdiscount')
                .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                    .select('plu_code' #items id
                            , 'rtl_txn_id' #cheques id
                            , 'base_qty'
                           )
               )

In [25]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes)) # только чеки с нужными plu

In [26]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [30]:
seg = (checks_tc5.select('customer_rk', 'group')
                  .distinct()
      )

[Stage 27:(10 + 21) / 210][Stage 28:> (8 + 4) / 156][Stage 29:> (7 + 8) / 159]9]

In [50]:
# seg.write.parquet('temp1066_1', mode='overwrite')
seg = spark.read.parquet('temp1066_1')

In [32]:
seg.count()

6420

In [45]:
client_segmentation = (spark.table('tc5_analytics_sanbox.t_TX_transform_client_segmentation')
                            .withColumnRenamed('curr_acc_no','account_no')
                            .filter(F.col('dt')==('2022-02-01'))
                            .select(['account_no','segment_person']))

In [46]:
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
client_segmentation = loyalty_cardholders.join(client_segmentation, on='account_no', how='inner')

In [47]:
client_segmentation = client_segmentation.select(['customer_rk', 'segment_person']).distinct()

In [36]:
seg_segmentation = seg.join(client_segmentation, 'customer_rk', 'inner')

In [37]:
seg_segmentation_pd = seg_segmentation.toPandas()

In [38]:
seg_segmentation_pd.head()

,customer_rk,segment_person
0,47793256,"46-60, не мало НЦС"
1,47793256,"46-60, не мало НЦС"
2,13689087,"46-60, не мало НЦС"
3,13689087,"46-60, не мало НЦС"
4,47362655,61+


In [39]:
groups = seg_segmentation_pd.groupby('segment_person')['customer_rk'].count().reset_index()

In [40]:
groups.head()

,segment_person,customer_rk
0,"21-, мало НЦС",62
1,"21-, не мало НЦС",108
2,"22-35, мало НЦС, есть дети",182
3,"22-35, мало НЦС, нет детей",630
4,"22-35, не мало НЦС, без вредных привычек",372


In [42]:
groups.to_excel('grouped_guests.xlsx', index=False)

### 2. Кто из гостей купил в период акции, хотя до этого перестал покупать.

In [9]:
start_date1 = datetime.date(2021, 6, 23)
end_date1 = datetime.date(2021, 10, 22)
start_date2 = datetime.date(2021, 10, 23)
end_date2 = datetime.date(2021, 12, 22)

In [10]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [11]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(guests_all, on='customer_rk', how='inner')

In [12]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [13]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsalnovat'
                               )
                   )


In [14]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [102, 103]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [15]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [16]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [17]:
category = (spark.table(PRODUCTS).withColumnRenamed('plu_id', 'plu_code')
            .select('plu_code', 'syntethic_category_id').distinct())

In [18]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [19]:
checks_tc5 = checks_tc5.join(category, 'plu_code', 'inner')

In [20]:
pdf = (checks_tc5.groupby('syntethic_category_id').agg(F.sum('base_qty').alias('base_qty'),
                                     F.sum('zsalnovat').alias('novat'),
                                     F.countDistinct('rtl_txn_id').alias('checks_qty'),
                                     F.countDistinct('account_no').alias('guests_qty')))

In [21]:
pdf.write.parquet('temp1066_2', mode='overwrite')
pdf = spark.read.parquet('temp1066_2')

In [22]:
summary = pdf.toPandas()

In [23]:
summary

,syntethic_category_id,base_qty,novat,checks_qty,guests_qty
0,102,991984.428,90774385.74,780624,202058
1,103,1267166.694,215715126.09,1347917,284171


In [90]:
summary.to_excel('summary_category.xlsx', index=False)

### 3. Анализ продаж магазинов в категории твердых сыров

In [6]:
only_target = guests_all.filter(F.col('group').isin(['tg1', 'tg2']))

In [7]:
start_date1 = datetime.date(2021, 12, 2)
end_date1 = datetime.date(2021, 12, 29)
start_date2 = datetime.date(2021, 9, 29)
end_date2 = datetime.date(2021, 12, 29)

#### Отбор магазинов

In [8]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [9]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(only_target, on='customer_rk', how='inner')

In [10]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [11]:
stores = checks_headers_tc5.select('store_id').distinct().toPandas()['store_id'].to_list()

#### Подсчет по бренду БЛ

In [32]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'rtl_txn_dt', 'store_id')
                     )


In [33]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(stores))

In [34]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')

In [35]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [66]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'zsalnovat'
                               )
                   )


In [67]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [103]
plu_brand_code = ['5330', 'D754']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [68]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [69]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [70]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [71]:
week1_start = datetime.date(2021, 12, 2)
week1_end = datetime.date(2021, 12, 8)
week2_start = datetime.date(2021, 12, 9)
week2_end = datetime.date(2021, 12, 15)
week3_start = datetime.date(2021, 12, 16)
week3_end = datetime.date(2021, 12, 22)
week4_start = datetime.date(2021, 12, 23)
week4_end = datetime.date(2021, 12, 29)

In [72]:
checks_tc5_with_week = (checks_tc5.withColumn('week', F.when(F.col('rtl_txn_dt').between(week1_start, week1_end), "week1")
                                             .when(F.col('rtl_txn_dt').between(week2_start, week2_end), "week2")
                                             .when(F.col('rtl_txn_dt').between(week3_start, week3_end), "week3")
                                             .when(F.col('rtl_txn_dt').between(week4_start, week4_end), "week4")
                                   ))

In [73]:
in_sum = (checks_tc5_with_week.groupby('week').agg(F.sum('zsalnovat').alias('sum_in_target_plu')))

In [74]:
in_sum_pd_target_plu = in_sum.toPandas()

In [75]:
in_sum_pd_target_plu

,week,sum_in_target_plu
0,week1,62056418.82
1,week2,66098553.57
2,week3,78179713.67
3,week4,91866727.78


In [76]:
in_sum_pd_target_plu.loc[0, 'start_dt'] = '02-12-2021'
in_sum_pd_target_plu.loc[0, 'end_dt'] = '08-12-2021'
in_sum_pd_target_plu.loc[1, 'start_dt'] = '09-12-2021'
in_sum_pd_target_plu.loc[1, 'end_dt'] = '15-12-2021'
in_sum_pd_target_plu.loc[2, 'start_dt'] = '16-12-2021'
in_sum_pd_target_plu.loc[2, 'end_dt'] = '22-12-2021'
in_sum_pd_target_plu.loc[3, 'start_dt'] = '23-12-2021'
in_sum_pd_target_plu.loc[3, 'end_dt'] = '29-12-2021'

In [77]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt
0,week1,62056418.82,02-12-2021,08-12-2021
1,week2,66098553.57,09-12-2021,15-12-2021
2,week3,78179713.67,16-12-2021,22-12-2021
3,week4,91866727.78,23-12-2021,29-12-2021


In [79]:
in_sum_pd_target_plu['prev'] = 0

In [80]:
for i in range(1, len(in_sum_pd_target_plu)):
    in_sum_pd_target_plu.loc[i, 'prev'] = in_sum_pd_target_plu.loc[i-1, 'sum_in_target_plu']

In [82]:
in_sum_pd_target_plu.loc[0, 'prev'] = in_sum_pd_target_plu.loc[0, 'sum_in_target_plu']

In [84]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt,prev
0,week1,62056418.82,02-12-2021,08-12-2021,62056418.82
1,week2,66098553.57,09-12-2021,15-12-2021,62056418.82
2,week3,78179713.67,16-12-2021,22-12-2021,66098553.57
3,week4,91866727.78,23-12-2021,29-12-2021,78179713.67


In [85]:
in_sum_pd_target_plu.loc[:, ['prev', 'sum_in_target_plu']] = in_sum_pd_target_plu[['prev', 'sum_in_target_plu']].astype('float')

In [86]:
in_sum_pd_target_plu['%_prev'] = ((in_sum_pd_target_plu['sum_in_target_plu'] - in_sum_pd_target_plu['prev']) * 100 / in_sum_pd_target_plu['prev'])

In [88]:
in_sum_pd_target_plu['prev'] = in_sum_pd_target_plu.sum_in_target_plu - in_sum_pd_target_plu.prev

In [89]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt,prev,%_prev
0,week1,"62,056,418.82",02-12-2021,08-12-2021,0.00,0.00
1,week2,"66,098,553.57",09-12-2021,15-12-2021,"4,042,134.75",6.51
2,week3,"78,179,713.67",16-12-2021,22-12-2021,"12,081,160.10",18.28
3,week4,"91,866,727.78",23-12-2021,29-12-2021,"13,687,014.11",17.51


In [91]:
in_sum_pd_target_plu.to_excel('in_sum_stores_brest_litovsk.xlsx', index=False)

#### Подсчет по целевым PLU

In [12]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'rtl_txn_dt', 'store_id')
                     )


In [13]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(stores))

In [14]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')

In [15]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [16]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'zsalnovat'
                               )
                   )


In [17]:
plu_codes_cat = [3367104,3388814,3672506,3919797,3926055,4046891,4046892,4046893,4118966,4137277]

In [18]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [19]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [20]:
week1_start = datetime.date(2021, 12, 2)
week1_end = datetime.date(2021, 12, 8)
week2_start = datetime.date(2021, 12, 9)
week2_end = datetime.date(2021, 12, 15)
week3_start = datetime.date(2021, 12, 16)
week3_end = datetime.date(2021, 12, 22)
week4_start = datetime.date(2021, 12, 23)
week4_end = datetime.date(2021, 12, 29)

In [21]:
checks_tc5_with_week = (checks_tc5.withColumn('week', F.when(F.col('rtl_txn_dt').between(week1_start, week1_end), "week1")
                                             .when(F.col('rtl_txn_dt').between(week2_start, week2_end), "week2")
                                             .when(F.col('rtl_txn_dt').between(week3_start, week3_end), "week3")
                                             .when(F.col('rtl_txn_dt').between(week4_start, week4_end), "week4")
                                   ))

In [22]:
in_sum = (checks_tc5_with_week.groupby('week').agg(F.sum('zsalnovat').alias('sum_in_target_plu')))

In [23]:
in_sum_pd_target_plu = in_sum.toPandas()

In [24]:
in_sum_pd_target_plu.loc[0, 'start_dt'] = '02-12-2021'
in_sum_pd_target_plu.loc[0, 'end_dt'] = '08-12-2021'
in_sum_pd_target_plu.loc[1, 'start_dt'] = '09-12-2021'
in_sum_pd_target_plu.loc[1, 'end_dt'] = '15-12-2021'
in_sum_pd_target_plu.loc[2, 'start_dt'] = '16-12-2021'
in_sum_pd_target_plu.loc[2, 'end_dt'] = '22-12-2021'
in_sum_pd_target_plu.loc[3, 'start_dt'] = '23-12-2021'
in_sum_pd_target_plu.loc[3, 'end_dt'] = '29-12-2021'

In [25]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt
0,NaN,NaN,02-12-2021,08-12-2021
1,NaN,NaN,09-12-2021,15-12-2021
2,NaN,NaN,16-12-2021,22-12-2021
3,NaN,NaN,23-12-2021,29-12-2021


In [26]:
in_sum_pd_target_plu['prev'] = 0

In [27]:
for i in range(1, len(in_sum_pd_target_plu)):
    in_sum_pd_target_plu.loc[i, 'prev'] = in_sum_pd_target_plu.loc[i-1, 'sum_in_target_plu']

In [28]:
in_sum_pd_target_plu.loc[0, 'prev'] = in_sum_pd_target_plu.loc[0, 'sum_in_target_plu']

In [29]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt,prev
0,NaN,NaN,02-12-2021,08-12-2021,NaN
1,NaN,NaN,09-12-2021,15-12-2021,NaN
2,NaN,NaN,16-12-2021,22-12-2021,NaN
3,NaN,NaN,23-12-2021,29-12-2021,NaN


In [30]:
in_sum_pd_target_plu.loc[:, ['prev', 'sum_in_target_plu']] = in_sum_pd_target_plu[['prev', 'sum_in_target_plu']].astype('float')

In [31]:
in_sum_pd_target_plu['%_prev'] = ((in_sum_pd_target_plu['sum_in_target_plu'] - in_sum_pd_target_plu['prev']) * 100 / in_sum_pd_target_plu['prev'])

In [32]:
in_sum_pd_target_plu['prev'] = in_sum_pd_target_plu.sum_in_target_plu - in_sum_pd_target_plu.prev

In [33]:
in_sum_pd_target_plu

,week,sum_in_target_plu,start_dt,end_dt,prev,%_prev
0,NaN,NaN,02-12-2021,08-12-2021,NaN,NaN
1,NaN,NaN,09-12-2021,15-12-2021,NaN,NaN
2,NaN,NaN,16-12-2021,22-12-2021,NaN,NaN
3,NaN,NaN,23-12-2021,29-12-2021,NaN,NaN


In [34]:
in_sum_pd_target_plu.to_excel('in_sum_stores_target_plu.xlsx', index=False)

#### Подсчет по бренду БЛ

In [35]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'rtl_txn_dt', 'store_id')
                     )


In [36]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(stores))

In [37]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(guests_all, on='customer_rk', how='inner')

In [38]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [39]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                                , 'zsalnovat'
                               )
                   )


In [40]:
plu_codes_cat = [3367104,3388814,3672506,3919797,3926055,4046891,4046892,4046893,4118966,4137277]

In [41]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [42]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [43]:
week1_start = datetime.date(2021, 12, 2)
week1_end = datetime.date(2021, 12, 8)
week2_start = datetime.date(2021, 12, 9)
week2_end = datetime.date(2021, 12, 15)
week3_start = datetime.date(2021, 12, 16)
week3_end = datetime.date(2021, 12, 22)
week4_start = datetime.date(2021, 12, 23)
week4_end = datetime.date(2021, 12, 29)

In [44]:
checks_tc5_with_week = (checks_tc5.withColumn('week', F.when(F.col('rtl_txn_dt').between(week1_start, week1_end), "week1")
                                             .when(F.col('rtl_txn_dt').between(week2_start, week2_end), "week2")
                                             .when(F.col('rtl_txn_dt').between(week3_start, week3_end), "week3")
                                             .when(F.col('rtl_txn_dt').between(week4_start, week4_end), "week4")
                                   ))

In [45]:
in_sum = (checks_tc5_with_week.groupby('week').agg(F.sum('zsalnovat').alias('sum_in_target_plu')))

In [47]:
in_sum_pd_target_plu = in_sum.toPandas()

In [ ]:
in_sum_pd_target_plu.loc[0, 'start_dt'] = '02-12-2021'
in_sum_pd_target_plu.loc[0, 'end_dt'] = '08-12-2021'
in_sum_pd_target_plu.loc[1, 'start_dt'] = '09-12-2021'
in_sum_pd_target_plu.loc[1, 'end_dt'] = '15-12-2021'
in_sum_pd_target_plu.loc[2, 'start_dt'] = '16-12-2021'
in_sum_pd_target_plu.loc[2, 'end_dt'] = '22-12-2021'
in_sum_pd_target_plu.loc[3, 'start_dt'] = '23-12-2021'
in_sum_pd_target_plu.loc[3, 'end_dt'] = '29-12-2021'

In [ ]:
in_sum_pd_target_plu

In [ ]:
in_sum_pd_target_plu['prev'] = 0

In [ ]:
for i in range(1, len(in_sum_pd_target_plu)):
    in_sum_pd_target_plu.loc[i, 'prev'] = in_sum_pd_target_plu.loc[i-1, 'sum_in_target_plu']

In [ ]:
in_sum_pd_target_plu.loc[0, 'prev'] = in_sum_pd_target_plu.loc[0, 'sum_in_target_plu']

In [ ]:
in_sum_pd_target_plu

In [ ]:
in_sum_pd_target_plu.loc[:, ['prev', 'sum_in_target_plu']] = in_sum_pd_target_plu[['prev', 'sum_in_target_plu']].astype('float')

In [ ]:
in_sum_pd_target_plu['%_prev'] = ((in_sum_pd_target_plu['sum_in_target_plu'] - in_sum_pd_target_plu['prev']) * 100 / in_sum_pd_target_plu['prev'])

In [ ]:
in_sum_pd_target_plu['prev'] = in_sum_pd_target_plu.sum_in_target_plu - in_sum_pd_target_plu.prev

In [ ]:
in_sum_pd_target_plu

In [ ]:
in_sum_pd_target_plu.to_excel('in_sum_stores_target_plu.xlsx', index=False)

### Средние чеки КГ и ЦГ в период до кампании и во время кампании

In [91]:
only_control = guests_all.filter(F.col('group').isin(['cg1', 'cg2']))

In [92]:
start_date1 = datetime.date(2021, 12, 16)
end_date1 = datetime.date(2021, 12, 22)
start_date2 = datetime.date(2021, 12, 23)
end_date2 = datetime.date(2021, 12, 29)

In [93]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [94]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(only_target, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(only_control, on='customer_rk', how='inner')

In [95]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [96]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [97]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [98]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [108]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [104]:
avg_checks_until_camp = (checks_tc5.groupby('account_no')
                         .agg((F.sum('zsale_vat') / F.countDistinct('rtl_txn_id')).alias('avg_checks'))
                         ).agg((F.sum('avg_checks') / F.countDistinct('account_no')).alias('avg_check'))

In [109]:
avg_checks_until_camp2 = (checks_tc52.groupby('account_no')
                         .agg((F.sum('zsale_vat') / F.countDistinct('rtl_txn_id')).alias('avg_checks'))
                         ).agg((F.sum('avg_checks') / F.countDistinct('account_no')).alias('avg_check'))

In [106]:
avg_checks_until_camp_pd = avg_checks_until_camp.toPandas()

In [110]:
avg_checks_until_camp_pd2 = avg_checks_until_camp2.toPandas()

In [111]:
avg_checks_until_camp_pd

,avg_check
0,657.53798804323


In [112]:
avg_checks_until_camp_pd2

,avg_check
0,652.50187005801


In [113]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [114]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(only_target, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(only_control, on='customer_rk', how='inner')

In [115]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [116]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [117]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [118]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [119]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [120]:
avg_checks_in_camp = (checks_tc5.groupby('account_no')
                         .agg((F.sum('zsale_vat') / F.countDistinct('rtl_txn_id')).alias('avg_checks'))
                         ).agg((F.sum('avg_checks') / F.countDistinct('account_no')).alias('avg_check'))

In [121]:
avg_checks_in_camp2 = (checks_tc52.groupby('account_no')
                         .agg((F.sum('zsale_vat') / F.countDistinct('rtl_txn_id')).alias('avg_checks'))
                         ).agg((F.sum('avg_checks') / F.countDistinct('account_no')).alias('avg_check'))

In [122]:
avg_checks_in_camp_pd = avg_checks_in_camp.toPandas()

In [126]:
avg_checks_in_camp_pd2 = avg_checks_in_camp2.toPandas()

In [124]:
avg_checks_in_camp_pd

,avg_check
0,735.18779490930


In [127]:
avg_checks_in_camp_pd2

,avg_check
0,723.03533688746


In [144]:
avg_checks = avg_checks_until_camp_pd.append(avg_checks_until_camp_pd2).append(avg_checks_in_camp_pd).append(avg_checks_in_camp_pd2).reset_index(drop=True)

In [145]:
avg_checks = avg_checks.astype('float')

In [146]:
avg_checks = round(avg_checks, 2)

In [148]:
avg_checks.loc[0, 'label'] = 'until_target'
avg_checks.loc[1, 'label'] = 'until_control'
avg_checks.loc[2, 'label'] = 'in_target'
avg_checks.loc[3, 'label'] = 'in_control'

In [150]:
avg_checks.loc[2, '+%'] = (avg_checks.iloc[2, 0] - avg_checks.iloc[0, 0]) * 100 / avg_checks.iloc[0, 0]
avg_checks.loc[3, '+%'] = (avg_checks.iloc[3, 0] - avg_checks.iloc[1, 0]) * 100 / avg_checks.iloc[1, 0]

In [151]:
avg_checks

,avg_check,label,+%
0,657.54,until_target,NaN
1,652.50,until_control,NaN
2,735.19,in_target,11.81
3,723.04,in_control,10.81


In [152]:
avg_checks.to_excel('avg_checks.xlsx', index=False)

### 5. Разложить по персонам тех, кому была коммуникация.

In [43]:
commun_second_camp = experiment_guests_communications.filter(F.col('is_communicated') == 1).select('customer_rk')

In [48]:
seg_segmentation_2_camp = commun_second_camp.join(client_segmentation, 'customer_rk', 'inner')

In [49]:
seg_segmentation_2_camp_pd = seg_segmentation_2_camp.toPandas()

In [56]:
seg_segmentation_1_camp = guests_tg1.join(client_segmentation, 'customer_rk', 'inner')

In [ ]:
seg_segmentation_1_camp_pd = seg_segmentation_1_camp.toPandas()

In [63]:
grouped_1 = seg_segmentation_1_camp_pd.groupby('segment_person')['customer_rk'].count().reset_index()
grouped_2 = seg_segmentation_2_camp_pd.groupby('segment_person')['customer_rk'].count().reset_index()

In [69]:
grouped_1.to_excel('grouped_camp1.xlsx', index=False)
grouped_2.to_excel('grouped_camp2.xlsx', index=False)

### 6. Еще раз отобрать гостей в категории сыров

##### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['churn', 'frequency', 'upgrade', 'cross', 'ump']
dt = datetime.date.today()

In [8]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )


In [9]:
print(customers_usecase.count()) #

[Stage 0:=============================>                            (5 + 5) / 10]22/02/21 14:43:31 ERROR cluster.YarnScheduler: Lost executor 2 on mn-hd0149.x5.ru: Container container_e1331_1645433512905_3051_01_000004 on host: mn-hd0149.x5.ru was preempted.
[Stage 0:====================================================>     (9 + 1) / 10]

49592598


### 1 проверяем на доступность отобранных гостей в определенную дату

In [74]:
check_date = '2022-04-01'

In [75]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [76]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [77]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [78]:
seg_sms4 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[3], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [79]:
seg_sms5 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[4], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [80]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).union(seg_sms4).union(seg_sms5).distinct()

In [10]:
# seg_sms.write.parquet('temp1066_seg_sms', mode='overwrite')
seg_sms = spark.read.parquet('temp1066_seg_sms')

In [83]:
seg_sms.count() #

25431517

In [84]:
slip1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [85]:
slip2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

22/02/17 15:22:41 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.58:42370 is closed


 Время выполнения: 0:00:02


In [86]:
slip3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [87]:
slip4 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[3], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [88]:
slip5 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[4], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [89]:
slip = slip1.union(slip2).union(slip3).union(slip4).union(slip5).distinct()

In [11]:
# slip.write.parquet('temp1066_slip', mode='overwrite')
slip = spark.read.parquet('temp1066_slip')

In [91]:
slip.count() #

44036344

## Отбор магазинов из таблицы Excel

In [12]:
stores = pd.read_excel('need_info_stores.xlsx')

In [13]:
stores = stores['financial_unit_id'].tolist()

In [14]:
len(stores)

2583

In [15]:
plants = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .filter(F.col('financial_unit_id').isin(stores))
            .selectExpr('store_id as plant')
            .distinct()
            .toPandas()['plant']
            .tolist()
            )

len(plants)

3010

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [16]:
start_date = datetime.date(2021, 11, 16)
end_date = datetime.date(2022, 2, 15)

In [17]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [18]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [19]:
checks_headers_tc52 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [20]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip, on='customer_rk', how='inner')

In [21]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [22]:
checks_headers_tc52 = checks_headers_tc52.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [23]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [24]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [103]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [25]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [26]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [27]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [28]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [29]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [30]:
seg2 = (checks_tc52.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [31]:
seg = seg1.union(seg2).distinct()

In [95]:
seg.write.parquet('temp1066_seg_all_cheese', mode='overwrite')
seg = spark.read.parquet('temp1066_seg_all_cheese')

In [96]:
seg.count()

4152460

In [76]:
customers_churn = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin([usecase[0]]))
                     .select('account_no', 'customer_rk')
                            )
customers_freq = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin([usecase[1]]))
                     .select('account_no', 'customer_rk')
                            )


In [98]:
seg_churn = seg.join(customers_churn, 'customer_rk', 'inner')
seg_freq = seg.join(customers_freq, 'customer_rk', 'inner')
seg_others = seg.join(seg_churn, 'customer_rk', 'left_anti').join(seg_freq, 'customer_rk', 'left_anti')

In [99]:
seg_churn_pd = seg_churn.toPandas()

In [100]:
seg_freq_pd = seg_freq.toPandas()

In [101]:
seg_others_pd = seg_others.toPandas()

In [106]:
seg_others_pd['customer_rk'] = seg_others_pd['customer_rk'].astype('str')

In [107]:
seg_churn_sms = seg_churn_pd[seg_churn_pd.group == 'sms']
seg_churn_slip = seg_churn_pd[seg_churn_pd.group == 'slip']
seg_freq_sms = seg_freq_pd[seg_freq_pd.group == 'sms']
seg_freq_slip = seg_freq_pd[seg_freq_pd.group == 'slip']
seg_others_sms = seg_others_pd[seg_others_pd.group == 'sms']
seg_others_slip = seg_others_pd[seg_others_pd.group == 'slip']

In [108]:
cross, upgrade, ump = get_usecases(seg_others_sms)

In [109]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(seg_churn_sms), len(seg_freq_sms), len(cross), len(upgrade), len(ump), len(seg_churn_sms) + len(seg_freq_sms) + len(cross)+len(upgrade)+len(ump)))

churn: 110800, frequency: 478115, cross: 332251, upgrade: 399312, ump: 157623 (all: 1478101)


In [110]:
cross, upgrade, ump = get_usecases(seg_others_slip)

In [111]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(seg_churn_slip), len(seg_freq_slip), len(cross), len(upgrade), len(ump), len(seg_churn_slip) + len(seg_freq_slip) + len(cross)+len(upgrade)+len(ump)))

churn: 187967, frequency: 758962, cross: 690200, upgrade: 689064, ump: 348166 (all: 2674359)


#### Отбор тех, кто 6 месяцев назад покупал сыр 4 месяца, но 2 месяца уже не покупает

In [32]:
start_date1 = datetime.date(2021, 8, 10)
end_date1 = datetime.date(2022, 11, 15)
start_date2 = datetime.date(2021, 11, 16)
end_date2 = datetime.date(2022, 2, 9)

In [33]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [45]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants))

In [46]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(slip, on='customer_rk', how='inner')

In [47]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [48]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [49]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [50]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [103]
# plu_brand_code = ['2452']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [51]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [52]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [53]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [54]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [55]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [56]:
seg2 = (checks_tc5.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [57]:
checks_headers3 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [58]:
checks_headers3 = checks_headers3.filter(F.col('store_id').isin(plants))

In [59]:
loyalty_cards3 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders3 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info3 = loyalty_cards3.join(loyalty_cardholders3, on='account_no', how='inner')
clients_info3 = clients_info3.join(seg_sms, on='customer_rk', how='inner')
clients_info4 = loyalty_cards3.join(loyalty_cardholders3, on='account_no', how='inner')
clients_info4 = clients_info4.join(slip, on='customer_rk', how='inner')

In [60]:
checks_headers_tc53 = checks_headers3.join(clients_info3, on='loyalty_card_no') #оставили чеки только нужных гостей

In [61]:
checks_headers_tc54 = checks_headers3.join(clients_info4, on='loyalty_card_no') #оставили чеки только нужных гостей

In [62]:
checks_items3 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [63]:
checks_items3 = checks_items3.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [64]:
checks_tc53 = checks_items3.join(checks_headers_tc53, 'rtl_txn_id', how='inner')

In [65]:
checks_tc54 = checks_items3.join(checks_headers_tc54, 'rtl_txn_id', how='inner')

In [66]:
seg3 = (checks_tc53.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [67]:
seg_no_cheese_1 = seg_sms.join(seg3, 'customer_rk', 'left_anti')

In [68]:
seg4 = (checks_tc54.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [69]:
seg_no_cheese_2 = slip.join(seg4, 'customer_rk', 'left_anti')

In [70]:
seg_1 = seg_no_cheese_1.join(seg1, 'customer_rk', 'inner')

In [71]:
seg_2 = seg_no_cheese_2.join(seg2, 'customer_rk', 'inner')

In [72]:
seg = seg_1.union(seg_2).distinct()

In [73]:
seg.write.parquet('temp1066_ottok_new', mode='overwrite')
seg = spark.read.parquet('temp1066_ottok_new')

In [74]:
seg.count()

1868840

In [78]:
seg_churn = seg.join(customers_churn, 'customer_rk', 'inner')
seg_freq = seg.join(customers_freq, 'customer_rk', 'inner')
seg_others = seg.join(seg_churn, 'customer_rk', 'left_anti').join(seg_freq, 'customer_rk', 'left_anti')

In [79]:
seg_churn_pd = seg_churn.toPandas()

In [80]:
seg_freq_pd = seg_freq.toPandas()

In [81]:
seg_others_pd = seg_others.toPandas()

In [89]:
seg_churn_sms = seg_churn_pd[seg_churn_pd.group == 'sms']
seg_churn_slip = seg_churn_pd[seg_churn_pd.group == 'slip']
seg_freq_sms = seg_freq_pd[seg_freq_pd.group == 'sms']
seg_freq_slip = seg_freq_pd[seg_freq_pd.group == 'slip']
seg_others_sms = seg_others_pd[seg_others_pd.group == 'sms']
seg_others_slip = seg_others_pd[seg_others_pd.group == 'slip']

In [87]:
seg_others_pd['customer_rk'] = seg_others_pd['customer_rk'].astype('str')

In [90]:
cross, upgrade, ump = get_usecases(seg_others_sms)

In [91]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(seg_churn_sms), len(seg_freq_sms), len(cross), len(upgrade), len(ump), len(seg_churn_sms) + len(seg_freq_sms) + len(cross)+len(upgrade)+len(ump)))

churn: 175486, frequency: 367737, cross: 151515, upgrade: 173622, ump: 72511 (all: 940871)


In [92]:
cross, upgrade, ump = get_usecases(seg_others_slip)

In [93]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(seg_churn_slip), len(seg_freq_slip), len(cross), len(upgrade), len(ump), len(seg_churn_slip) + len(seg_freq_slip) + len(cross)+len(upgrade)+len(ump)))

churn: 164815, frequency: 365547, cross: 151502, upgrade: 173603, ump: 72502 (all: 927969)
